# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=["price"])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Fit the model and apply the model to the make test set predictions

In [5]:
from sklearn.linear_model import LinearRegression
linereg = LinearRegression()
linereg.fit(X_train, y_train)
yhat_train = linereg.predict(X_train)
yhat_test = linereg.predict(X_test)

Calculate the residuals and the mean squared error

In [6]:
res_train = yhat_train-y_train
mse_train = np.sum(res_train**2)/len(y_train)
res_test = yhat_test-y_test
mse_test = np.sum(res_test**2)/len(y_test)
mse_test,mse_train

(price    19.410589
 dtype: float64, price    22.55928
 dtype: float64)

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [7]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    folds = []
    fold_size = len(data)//k
    remainder = len(data)%k
    start = 0
    for fold in range(1,k+1):
        if fold<= remainder:
            folds.append(data.iloc[start:(start+fold_size+1)])
            start+=(fold_size +1)
        else:    
            folds.append(data.iloc[start:(start+fold_size)])
            start+=fold_size
    # add 1 to fold size to account for leftovers
    return folds

### Apply it to the Boston Housing Data

In [8]:
# Make sure to concatenate the data again
boston_data= pd.concat([X, y], axis=1)
boston_data.head()

,CHAS,RM,DIS,B,LSTAT,price
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [9]:
boston_folds = kfolds(boston_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [10]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i,fold in enumerate(boston_folds) if i!=n])
    test = boston_folds[n]
    # Fit a linear regression model
    linereg.fit(train[X.columns], train.price)
    #Evaluate Train and Test Errors
    yhat_train = linereg.predict(train[X.columns])
    yhat_test = linereg.predict(test[X.columns])
    res_train = yhat_train-train.price
    mse_train = np.sum(res_train**2)/len(train.price)
    train_errs.append(mse_train)
    res_test = yhat_test-test.price
    mse_test = np.sum(res_test**2)/len(test.price)
    test_errs.append(mse_test)
print(train_errs)
print(test_errs)

[24.195577370388616, 23.032087348477965, 19.74507285797898, 15.31710138425119, 22.32997280754659]
[13.405144922008212, 17.444016800682025, 37.03271139002075, 58.27954384784233, 26.097988757148386]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linereg, X, y, cv=5, scoring="neg_mean_squared_error")
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [14]:
np.mean(cv_5_results)

-30.451881143540323

The cross validation score has a higher value than the train_test_split error value.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!